## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

In [2]:
from fastembed import TextEmbedding

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51


## Q1 SOLUTION

In [3]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=3))

{
   "model": "BAAI/bge-small-zh-v1.5",
   "sources": {
      "hf": "Qdrant/bge-small-zh-v1.5",
      "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
      "_deprecated_tar_struct": true
   },
   "model_file": "model_optimized.onnx",
   "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
   "license": "mit",
   "size_in_GB": 0.09,
   "additional_files": [],
   "dim": 512,
   "tasks": {}
}
{
   "model": "Qdrant/clip-ViT-B-32-text",
   "sources": {
      "hf": "Qdrant/clip-ViT-B-32-text",
      "url": null,
      "_deprecated_tar_struct": false
   },
   "model_file": "model.onnx",
   "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
   "license": "mit",
   "size_in_GB": 0.25,
   "additional_files": [],
   "dim": 512,
   "tasks": {}
}
{
   "model": "

In [4]:
# Load the Jina embedding model
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Your input text
query1 = 'I just discovered the course. Can I join now?'

# Get embeddings (returns a generator, so convert to list)
embeddings = list(model.embed(query1))
embedding_vector = embeddings[0]

# Optional: Get dimensionality
EMBEDDING_DIMENSIONALITY = len(embedding_vector)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:01, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [5]:
min(embedding_vector)

np.float64(-0.11726373885183883)

In [6]:
EMBEDDING_DIMENSIONALITY

512

## Q1 ANSWER
b) -0.11

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9

## Q2 SOLUTION

In [7]:
import numpy as np
np.linalg.norm(embedding_vector)

np.float64(1.0)

In [11]:
doc = 'Can I still join the course after the start date?'
# Get embeddings (returns a generator, so convert to list)
doc_embeddings = list(model.embed(doc))
doc_embedding_vector = doc_embeddings[0]

In [14]:
doc_embedding_vector.dot(embedding_vector)

np.float64(0.9008528895674548)

## Q2 ANSWER
d) 0.9

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

In [30]:
emb_list = []
for doc in documents:
    doc_embeddings = list(model.embed(doc['text']))
    doc_embedding_vector = doc_embeddings[0]
    emb_list.append(doc_embedding_vector)
V = np.vstack([emb_list[0], emb_list[1], emb_list[2] , emb_list[3], emb_list[4]])

In [31]:
V.dot(embedding_vector)

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73712505])

## Q3 ANSWER
d) the closest distance is 0.71330788 for index 3

In [12]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [13]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [14]:
# Define the collection name
collection_name = "homework2"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [15]:
query1 = 'I just discovered the course. Can I join now?'

In [17]:
type(query1)

str

In [18]:
point = models.PointStruct(
            id=0,
            vector=models.Document(text=query1, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        )

In [22]:
client.upsert(
    collection_name=collection_name,
    points=[point]
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
models.Vector

typing.Union[typing.List[typing.Annotated[float, Strict(strict=True)]], qdrant_client.http.models.models.SparseVector, typing.List[typing.List[typing.Annotated[float, Strict(strict=True)]]], qdrant_client.http.models.models.Document, qdrant_client.http.models.models.Image, qdrant_client.http.models.models.InferenceObject]

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```